<a href="https://colab.research.google.com/github/Ali-Hasan-Khan28/Natural-Language-Processing/blob/main/Lab11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.decomposition import PCA
from transformers import RobertaTokenizer, TFRobertaModel
import random

In [ ]:
def generate_large_dataset(base_data, num_samples):
    augmented_data = []
    for _ in range(num_samples):
        base_sample_index = random.randint(0, len(base_data) - 1)
        base_sample = base_data.iloc[base_sample_index]
        sentiment = random.choice(['Positive', 'Negative', 'Neutral'])
        emotions = random.sample(['Joy', 'Sadness', 'Anger', 'Fear', 'Surprise', 'Indifference'], k=random.randint(1, 3))
        augmented_sample = {
            'text': base_sample['text'],
            'sentiment': sentiment,
            'emotion': emotions
        }
        augmented_data.append(augmented_sample)
    return pd.DataFrame(augmented_data)

In [ ]:
data = pd.DataFrame({
    'text': ["I absolutely love the new batman movie!",
             "I'm really not looking forward to monday",
             "This is just an average day for me"],
    'sentiment': ['Positive', 'Negative', 'Neutral'],
    'emotion': [['Joy', 'Excitement'], ['Sadness', 'Dread'], ['Indifference']]
})


In [ ]:
larger_dataset = generate_large_dataset(data, 1000)

In [ ]:
def create_roberta_model():
    input_ids = tf.keras.layers.Input(shape=(512,), dtype=tf.int32)
    attention_mask = tf.keras.layers.Input(shape=(512,), dtype=tf.int32)
    roberta = TFRobertaModel.from_pretrained('roberta-base')
    roberta_output = roberta({'input_ids': input_ids, 'attention_mask': attention_mask})[0]
    sentiment_output = tf.keras.layers.Dense(3, activation='softmax', name='sentiment_output')(roberta_output[:, 0, :])
    emotion_output = tf.keras.layers.Dense(6, activation='softmax', name='emotion_output')(roberta_output[:, 0, :])
    model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=[sentiment_output, emotion_output])
    return model

In [ ]:
train_data, test_data = train_test_split(larger_dataset, test_size=0.2, random_state=42)

# Tokenize input texts
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
train_encodings = tokenizer(list(train_data['text']), truncation=True, padding='max_length', max_length=512, return_tensors='tf')
test_encodings = tokenizer(list(test_data['text']), truncation=True, padding='max_length', max_length=512, return_tensors='tf')

# Prepare labels
train_sentiment_labels = np.array([['Positive', 'Negative', 'Neutral'].index(label) for label in train_data['sentiment']])
test_sentiment_labels = np.array([['Positive', 'Negative', 'Neutral'].index(label) for label in test_data['sentiment']])
train_emotion_labels = np.array([[1 if emotion in label else 0 for emotion in ['Joy', 'Sadness', 'Anger', 'Fear', 'Surprise', 'Indifference']] for label in train_data['emotion']])
test_emotion_labels = np.array([[1 if emotion in label else 0 for emotion in ['Joy', 'Sadness', 'Anger', 'Fear', 'Surprise', 'Indifference']] for label in test_data['emotion']])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

In [ ]:
model = create_roberta_model()

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss={'sentiment_output': 'sparse_categorical_crossentropy', 'emotion_output': 'binary_crossentropy'},
              metrics={'sentiment_output': 'accuracy', 'emotion_output': 'accuracy'})

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaModel: ['lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'roberta.embeddings.position_ids', 'lm_head.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing TFRobertaModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaModel were not initialized from the PyTorch model and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and infe

In [ ]:
history = model.fit([train_encodings['input_ids'], train_encodings['attention_mask']],
                    {'sentiment_output': train_sentiment_labels, 'emotion_output': train_emotion_labels},
                    validation_data=([test_encodings['input_ids'], test_encodings['attention_mask']],
                                     {'sentiment_output': test_sentiment_labels, 'emotion_output': test_emotion_labels}),
                    epochs=3, batch_size=8)


Epoch 1/3


 45/100 [============>.................] - ETA: 48:57 - loss: 1.8074 - sentiment_output_loss: 1.1302 - emotion_output_loss: 0.6772 - sentiment_output_accuracy: 0.3222 - emotion_output_accuracy: 0.1889

In [ ]:
test_predictions = model.predict([test_encodings['input_ids'], test_encodings['attention_mask']])
sentiment_predictions = np.argmax(test_predictions[0], axis=1)
emotion_predictions = np.argmax(test_predictions[1], axis=1)

In [ ]:
sentiment_accuracy = accuracy_score(test_sentiment_labels, sentiment_predictions)
emotion_accuracy = accuracy_score(test_emotion_labels.argmax(axis=1), emotion_predictions)

In [ ]:
print("Sentiment Accuracy:", sentiment_accuracy)
print("Emotion Accuracy:", emotion_accuracy)